<a href="https://colab.research.google.com/github/jnrkufuor/apollo/blob/Ryan/NER_RH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Packages

In [1]:
# install flair
!pip install flair

# load basic packages
import pandas as pd
from itertools import combinations, product
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# load Flair and NLTK
import torch
from flair.data import Sentence
from flair.models import SequenceTagger
from nltk import tokenize

import nltk
nltk.download('punkt')

# is cuda available?
torch.cuda.is_available()

     |████████████████████████████████| 286kB 21.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 798kB 45.2MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 1.2MB 47.7MB/s 
     |████████████████████████████████| 19.7MB 1.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.1MB 50.1MB/s 
     |████████████████████████████████| 983kB 48.8MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 890kB 49.1MB/s 
     |████████████████████████████████| 3.2MB 50.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=04a93ab760d643e92b1ddf687f9e898f5528aa776018945eb7c2db1e7d30d64b
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Succes

True

## 2. Load Flair NER model

In [2]:
#Load NER Model
tagger = SequenceTagger.load('ner')

2021-03-16 18:16:27,783 --------------------------------------------------------------------------------
2021-03-16 18:16:27,784 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-03-16 18:16:27,786  - The most current version of the model is automatically downloaded from there.
2021-03-16 18:16:27,787  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-03-16 18:16:27,788 --------------------------------------------------------------------------------



2021-03-16 18:16:49,084 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


## 3. Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/googlenews1000.csv')
#Above two lines may have to be changed to import your data depending on where it is
df

Mounted at /content/drive


,Unnamed: 0,title,media,date,datetime,desc,link,img,content
0,0,STUCK IN WALMART PARKING lot after Colorado storm,FOX31 Denver,1 hour ago,2021-03-15 22:26:32.753586,Several drivers were forced into a Walmart par...,https://kdvr.com/news/local/drivers-spend-hour...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","AURORA, Colo. (KDVR) — The historic snowstorm ..."
1,1,Suspect wanted in fatal Monkey Junction Walmar...,Home - WSFX,1 hour ago,2021-03-15 22:26:32.755743,Laron Lee Carter is wanted by the NHC Sheriff'...,https://foxwilmington.com/local-news/suspect-w...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",According to a spokesperson for the New Hanove...
2,2,Walmart Theft Investigation Turns Into Chase A...,KVRR,1 hour ago,2021-03-15 22:26:32.757773,— Two people are arrested after a chase where ...,https://www.kvrr.com/2021/03/15/walmart-theft-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Walmart Theft Investigation Turns Into Chase A...
3,3,Walmart Calls Ex-Worker's 'Stonewalling' Claim...,NaN,1 hour ago,2021-03-15 22:26:32.759685,"Law360 (March 15, 2021, 5:49 PM EDT) -- An ex-...",https://www.law360.com/retail/articles/1364916...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Law360 (March 15, 2021, 5:49 PM EDT) -- An ex-..."
4,4,Three wanted for questioning in Walmart shopli...,wgxa.tv,1 hour ago,2021-03-15 22:26:32.761621,"HOUSTON COUNTY, Ga. -- Perry police are lookin...",https://wgxa.tv/news/local/three-wanted-for-qu...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",NaN
...,...,...,...,...,...,...,...,...,...
994,9,Programmable Metallization Cell Market 2021 In...,NaN,8 hours ago,2021-03-15 15:28:20.158326,Axon Technologies; Micron Technology; Fujitsu ...,https://soccernurds.com/uncategorized/2268339/...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Industry Growth Insights (IGI) has published a...
995,0,Tampa International Airport sees spike in trav...,WFLA,16 mins ago,2021-03-15 23:12:21.719311,This past Saturday airport officials say they ...,https://www.wfla.com/community/health/coronavi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","TAMPA, Fla. (WFLA) – People are taking flight ..."
996,1,Local Wis. health officials urge against trave...,WMTV - NBC15,46 mins ago,2021-03-15 22:42:21.723116,"MADISON, Wis. (WMTV) - With spring break right...",https://www.nbc15.com/2021/03/15/local-wis-hea...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","MADISON, Wis. (WMTV) - With spring break right..."
997,2,Oklahomans traveling for spring break despite ...,kjrh.com,50 mins ago,2021-03-15 22:38:21.726876,"TULSA, Okla. — 2020 was a financial strain for...",https://www.kjrh.com/news/local-news/spring-br...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","TULSA, Okla. — 2020 was a financial strain for..."


#4. Remove pronouns

In [4]:

pronouns = ['I', 'You', 'It', 'He', 'She', 'We', 'They']
suffixes = ["", "’m", "’re", "’s", "’ve", "’d", "'m", "'re", "'s", "'ve", "'d", "m", "re", "s", "ve", "d"]

contraptions = [(p, s) for p in pronouns for s in suffixes]

df_contraptions = pd.DataFrame(contraptions, columns=['pronoun', 'suffix'])

df_contraptions['contraption'] = df_contraptions.apply(lambda x: x['pronoun'] + x['suffix'], axis=1)

contraptions = df_contraptions.contraption.values


## 4. Define NER function

In [22]:
# define function

def get_ner_data(paragraph):
    '''
    - function to extract named entities from a paragraph
    - returns two data frames:
        - the first is a dataframe of all unique entities (persons and orgs)
        - the second is the links between the entities
    '''
    
    # remove newlines and odd characters
    paragraph = re.sub('\r', '', paragraph)
    paragraph = re.sub('\n', ' ', paragraph)
    paragraph = re.sub("’s", '', paragraph)
    paragraph = re.sub("“", '', paragraph)
    paragraph = re.sub("”", '', paragraph)

    
    # tokenise sentences
    sentences = tokenize.sent_tokenize(paragraph)
    sentences = [Sentence(sent) for sent in sentences]
    
    # predict named entities
    for sent in sentences:
        tagger.predict(sent)
    
    # collect sentence NER's to list of dictionaries
    sent_dicts = [sentence.to_dict(tag_type='ner') for sentence in sentences]
    
    # collect entities and types
    entities = []
    types = []
    for sent_dict in sent_dicts:
        entities.extend([entity['text'] for entity in sent_dict['entities']])
        types.extend([str(entity['labels'])[1:4] for entity in sent_dict['entities']])
   #The above line is what I changed from the default notebook to get things working     
    
    # create dataframe of entities (nodes)
    df_ner = pd.DataFrame(data={'entity': entities, 'type': types})
    df_ner = df_ner[df_ner['type'].isin(['PER','ORG'])]
    df_ner = df_ner[df_ner['entity'].map(lambda x: isinstance(x, str))]
    df_ner = df_ner[~df_ner['entity'].isin(df_contraptions['contraption'].values)]
    df_ner['entity'] = df_ner['entity'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    df_ner['entity'] = df_ner.apply(lambda x: x['entity'].split(' ')[len(x['entity'].split(' '))-1] if x['type']=='PER' else x['entity'], axis=1)
    df_ner = df_ner.drop_duplicates().sort_values('entity')
    
    # get entity combinations
    combs = list(combinations(df_ner['entity'], 2))
    
    # create dataframe of relationships (edges)
    df_links = pd.DataFrame(data=combs, columns=['from', 'to'])
    
    return df_ner, df_links

## 5. Apply function

In [ ]:
#df['date'] = pd.to_datetime(df['date'])
#day = '2021-01-24'
#df_day = df[df['date']==pd.to_datetime(day)][['date', 'domain', 'title', 'content']]


In [6]:
df_domain = df.groupby('media').agg({'content': 'count'}).reset_index()
df_domain.columns = ['media', 'count']
df_domain = df_domain.sort_values('count', ascending=False)
dfd_small=df_domain.iloc[1:21,:]

dfd_small

# g2 = sns.barplot(data=dfd_small,
#              x='count',
#              y='Domain',
#              dodge=False,
#              orient='h',
#              hue='count',
#              palette='viridis')

# g2.set_yticks([])
# g2.set_title('Number of articles from each provider')
# g2.set_xlabel('Count')
# g2.set_ylabel('')
# g2.set_xlim(0, max(dfd_small['count'])+150)
# g2.legend_.remove()
# g2.tick_params(labelsize=5)

# for i in dfd_small.index:
#             g2.text(df_domain.iloc[i]['count']+5, i+0.25, df_domain.iloc[i]['Domain'], fontsize=8)

# sns.despine()
# g2.get_figure().savefig('domain_plot.png', dpi=1000)

,media,count
233,SoccerNurds,23
325,Yahoo Finance,22
183,NeighborWebSJ,22
251,The Bisouv Network,14
21,Bloomberg,8
236,State Reviewer,8
172,Murphy's Hockey Law,7
253,The Courier,7
250,The Baxter Report,7
200,Patch.com,6


In [25]:
df_ner = pd.DataFrame()
df_links = pd.DataFrame()

for content in tqdm(df['content']):
  try:
    df_ner_temp, df_links_temp = get_ner_data(content)

    df_ner = df_ner.append(df_ner_temp)
    df_links = df_links.append(df_links_temp)
  except:
    continue



100%|██████████| 999/999 [38:54<00:00,  2.34s/it]


In [26]:
# praph=df['content'].iloc[1]
# praph = re.sub('\r', '', praph)
# praph = re.sub('\n', ' ', praph)
# praph = re.sub("’s", '', praph)
# praph = re.sub("“", '', praph)
# praph = re.sub("”", '', praph)

    
# # tokenise sentences
# sentences = tokenize.sent_tokenize(praph)
# sentences = [Sentence(sent) for sent in sentences]

# ## predict named entities
# for sent in sentences:
#     tagger.predict(sent)
    
# # # collect sentence NER's to list of dictionaries
# sent_dicts = [sentence.to_dict(tag_type='ner') for sentence in sentences]



# # # collect entities and types
# entities = []
# types = []
# for sent_dict in sent_dicts:
#     entities.extend([entity['text'] for entity in sent_dict['entities']])
#     types.extend([str(entity['labels'])[1:4] for entity in sent_dict['entities']])

# types

,entity,type
11,Cabrera,PER
8,Stokes,PER
13,Thomas,PER
15,Uber,PER
9,Volvo,ORG
...,...,...
2,Tulsa International Airport,ORG
5,Denny,PER
4,East Central Region AAA,ORG
6,Miller Travel,ORG


# 6. Remove plurals and possessives

In [27]:
def remove_s(entity, entity_series):
  if (entity[-1] == 's') & (entity[:-1] in entity_series):
    return entity[:-1]
  else:
    return entity



In [29]:
df_links['to'] = df_links['to'].map(lambda x: remove_s(x, df_ner['entity'].values))
df_links['from'] = df_links['from'].map(lambda x: remove_s(x, df_ner['entity'].values))
df_ner['entity_cl'] = df_ner['entity'].map(lambda x: remove_s(x, df_ner['entity'].values))



,from,to
0,Cabrera,Stokes
1,Cabrera,Thomas
2,Cabrera,Uber
3,Cabrera,Volvo
4,Cabrera,Walmart
...,...,...
1,Denny,Miller Travel
2,Denny,William
3,East Central Region AAA,Miller Travel
4,East Central Region AAA,William


In [31]:
df_links[df_links['to'].str.contains('They')]

,from,to


# 7. Export Data

In [33]:
#df_ner.to_csv('/content/drive/My Drive/df_ner_att1.csv', index=False)
#df_links.to_csv('/content/drive/My Drive/df_links_att1.csv', index=False)

#Use the above two lines to write the critical dataframes to csv files in your 
#google drive account